In [1]:
from igsutils import IGSDownloader, IGSFileManager
from datetime import datetime, timezone
from types import SimpleNamespace
import re

there are 27 leap second epochs


In [2]:
file_manager = IGSFileManager('/media/share/brianbw/data/igs/',
                              '/gnss/data/daily/{yyyy}/{ddd}/{yy}n/', 'arev{ddd}0.{yy}n.Z')
downloader = IGSDownloader(file_manager, ftp_host='cddis.gsfc.nasa.gov', overwrite=False)

In [3]:
dt = datetime(2013, 2, 22, tzinfo=timezone.utc)
filepath = downloader.fetch(dt)

In [4]:
!du -hs {filepath}

120K	/media/share/brianbw/data/igs/gnss/data/daily/2013/053/13n/arev0530.13n


In [5]:
!head -n 25 {filepath}

     2.10           N: GPS NAV DATA                         RINEX VERSION / TYPE
teqc  2012Jun6      gpsops              20130223 00:10:08UTCPGM / RUN BY / DATE
    16                                                      LEAP SECONDS
Linux 2.4.21-27.ELsmp|Opteron|gcc -static|Linux x86_64|=+   COMMENT
    1.6764D-08  0.0000D+00 -1.1921D-07  5.9605D-08          ION ALPHA
    1.2288D+05 -3.2768D+04 -1.9661D+05 -6.5536D+04          ION BETA
    9.313225746155D-10 8.881784197001D-15   589824     1728 DELTA-UTC: A0,A1,T,W
AREV                                    MARKER NAME         COMMENT
42202M005                               MARKER NUMBER       COMMENT
1942826.192 -5804070.331 -1796894.267                       COMMENT
                                                            COMMENT
This data is provided as a public service by NASA/JPL.      COMMENT
No warranty is expressed or implied regarding suitability   COMMENT
for use.  For further information, contact:                 COMMENT
Da

In [6]:
'([+-]?\s?\d\.\d{12}D[+-]?\d{2})' * 3

'([+-]?\\s?\\d\\.\\d{12}D[+-]?\\d{2})([+-]?\\s?\\d\\.\\d{12}D[+-]?\\d{2})([+-]?\\s?\\d\\.\\d{12}D[+-]?\\d{2})'

In [7]:
# pattern = '(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+\.\d)([+-\s]?\d\.\d{12}D[+-]?\d{2})'
epoch_pattern = '(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+\.\d)'
number_pattern = '\n?\s*([+-]?\d+\.\d{12}D[+-]?\d{2})'
pattern = epoch_pattern + 29 * number_pattern

In [8]:
def parse_rinex_nav(filepath, century=2000):
    '''
    Given filepath to RINEX Navigation file, parses navigation into ephemeris.
    Returns dictionary {prn: [SimpleNamespace]} of ephemeris objects
    
    # (2016/05/30) function requiring ephemeris paramters need the following names:
    # e - eccentricity
    # t_oe - time of ephemeris
    # i_0 - inclination at reference time (rad)
    # a - semi-major axis (m); usually given as SQRT
    # omega_dot - rate of right ascension (rad/s)
    # omega_0 - right ascension at week (rad)
    # omega - argument of perigee
    # M_0 - mean anomaly of reference time (rad)
    # week - GPS week number
    # delta_n - mean motion difference (rad/s)
    # i_dot - rate of inclination angle (rad/s)
    # c_us - argument of latitude (amplitude of cosine, radians)
    # c_rs - orbit radius (amplitude of sine, meters)
    # c_is - inclination (amplitude of sine, meters)
    # c_uc - argument of latitude (amplitude of cosine, radians)
    # c_rc - orbit radius (amplitude of cosine, meters)
    # c_ic - inclination (amplitude of cosine, meters)century = 2000
    '''
    epoch_pattern = '(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+)\s(\s?\d+\.\d)'
    number_pattern = '\n?\s*([+-]?\d+\.\d{12}D[+-]?\d{2})'
    pattern = epoch_pattern + 29 * number_pattern
    data = {}
    with open(filepath, 'r') as f:
        matches = re.findall(pattern, f.read())
        for m in matches:
            prn, yy, month, day, hour, minute = (int(i) for i in m[:6])
            second, a0, a1, a2, \
                iode1, c_rs, delta_n, m_0, \
                c_uc, e, c_us, sqrt_a, \
                t_oe, c_ic, omega_0, c_is, \
                i_0, c_rc, omega, omega_dot, \
                i_dot, l2_codes, week, l2p_data, \
                accuracy, health, tgd, iodc, \
                transmit_time, fit_interval = (float(s.replace('D', 'E')) for s in m[6:36])
            year = century + yy
            epoch = datetime(year, month, day, hour, minute, int(second), int(1e6 * (second % 1)), tzinfo=timezone.utc)
            eph = SimpleNamespace(
                epoch=epoch, a0=a0, a1=a1, a2=a2, iodc=iodc,
                iode1=iode1, c_rs=c_rs, delta_n=delta_n, m_0=m_0,
                c_uc=c_uc, e=e, c_us=c_us, sqrt_a=sqrt_a,
                t_oe=t_oe, c_ic=c_ic, omega_0=omega_0, c_is=c_is,
                i_0=i_0, c_rc=c_rc, omega=omega, omega_dot=omega_dot,  # TODO check if orbit solutions correct omega
                i_dot=i_dot, l2_codes=l2_codes, week=week, l2p_data=l2p_data,
                accuracy=accuracy, health=health, tgd=tgd, iodc=iodc,
                transmit_time=transmit_time, fit_interval=fit_interval
            )
            if prn not in data.keys():
                data[prn] = []
            data[prn].append(eph)
    return data

In [9]:
data = parse_rinex_nav(filepath)

In [10]:
float(m[7].replace('D','E'))

NameError: name 'm' is not defined

See [here](https://igscb.jpl.nasa.gov/igscb/data/format/rinex210.txt) for RINEX NAV file format (2/3 of the way down).
```
 +----------------------------------------------------------------------------+
 |                                  TABLE A4                                  |
 |           GPS NAVIGATION MESSAGE FILE - DATA RECORD DESCRIPTION            |
 +--------------------+------------------------------------------+------------+
 |    OBS. RECORD     | DESCRIPTION                              |   FORMAT   |
 +--------------------+------------------------------------------+------------+
 |PRN / EPOCH / SV CLK| - Satellite PRN number                   |     I2,    |
 |                    | - Epoch: Toc - Time of Clock             |            |
 |                    |          year (2 digits, padded with 0   |            |
 |                    |                if necessary)             |  1X,I2.2,  |
 |                    |          month                           |   1X,I2,   |
 |                    |          day                             |   1X,I2,   |
 |                    |          hour                            |   1X,I2,   |
 |                    |          minute                          |   1X,I2,   |
 |                    |          second                          |    F5.1,   |
 |                    | - SV clock bias       (seconds)          |  3D19.12   |
 |                    | - SV clock drift      (sec/sec)          |            |
 |                    | - SV clock drift rate (sec/sec2)         |     *)     |
 +--------------------+------------------------------------------+------------+
 | BROADCAST ORBIT - 1| - IODE Issue of Data, Ephemeris          | 3X,4D19.12 |
 |                    | - Crs                 (meters)           |            |
 |                    | - Delta n             (radians/sec)      |            |
 |                    | - M0                  (radians)          |            |
 +--------------------+------------------------------------------+------------+
 | BROADCAST ORBIT - 2| - Cuc                 (radians)          | 3X,4D19.12 |
 |                    | - e Eccentricity                         |            |
 |                    | - Cus                 (radians)          |            |
 |                    | - sqrt(A)             (sqrt(m))          |            |
 +--------------------+------------------------------------------+------------+
 | BROADCAST ORBIT - 3| - Toe Time of Ephemeris                  | 3X,4D19.12 |
 |                    |                       (sec of GPS week)  |            |
 |                    | - Cic                 (radians)          |            |
 |                    | - OMEGA               (radians)          |            |
 |                    | - CIS                 (radians)          |            |
 +--------------------+------------------------------------------+------------+
 | BROADCAST ORBIT - 4| - i0                  (radians)          | 3X,4D19.12 |
 |                    | - Crc                 (meters)           |            |
 |                    | - omega               (radians)          |            |
 |                    | - OMEGA DOT           (radians/sec)      |            |
 +--------------------+------------------------------------------+------------+
 | BROADCAST ORBIT - 5| - IDOT                (radians/sec)      | 3X,4D19.12 |
 |                    | - Codes on L2 channel                    |            |
 |                    | - GPS Week # (to go with TOE)            |            |
 |                    |   Continuous number, not mod(1024)!      |            |
 |                    | - L2 P data flag                         |            |
 +--------------------+------------------------------------------+------------+
 | BROADCAST ORBIT - 6| - SV accuracy         (meters)           | 3X,4D19.12 |
 |                    | - SV health        (bits 17-22 w 3 sf 1) |            |
 |                    | - TGD                 (seconds)          |            |
 |                    | - IODC Issue of Data, Clock              |            |
 +--------------------+------------------------------------------+------------+
 | BROADCAST ORBIT - 7| - Transmission time of message       **) | 3X,4D19.12 |
 |                    |         (sec of GPS week, derived e.g.   |            |
 |                    |    from Z-count in Hand Over Word (HOW)  |            |
 |                    | - Fit interval        (hours)            |            |
 |                    |         (see ICD-GPS-200, 20.3.4.4)      |            |
 |                    |   Zero if not known                      |            |
 |                    | - spare                                  |            |
 |                    | - spare                                  |            |
 +--------------------+------------------------------------------+------------+
```

In [ ]:
print(len(matches))